In [17]:
import random
import simpy
import numpy as np

## Car Example
Modified from the example for the official simpy [website](https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html). 

- Random parking/trip time (lognormal)
- Print '.2%f'
- Add time units

In [2]:
PARK_MU = 2
PARK_SIGMA = 1
TRIP_MU = 0.1
TRIP_SIGMA = 1
RANDOM_SEED = 123
RUN_TIME = 100
TIME_UNIT = "hours"

def car(env):
     while True:
         print('Start parking at %.2f %s' % (env.now, TIME_UNIT))
         parking_duration = random.lognormvariate(PARK_MU, PARK_SIGMA)
         print('Parking for %.2f %s' % (parking_duration, TIME_UNIT))
         yield env.timeout(parking_duration)

         print('Start driving at %.2f %s' % (env.now, TIME_UNIT))
         trip_duration = random.lognormvariate(TRIP_MU, TRIP_SIGMA)
         print('Driving for %.2f %s' % (trip_duration, TIME_UNIT))
         yield env.timeout(trip_duration)

print('Machine shop')
random.seed(RANDOM_SEED)  
env = simpy.Environment()
env.process(car(env))
env.run(until = RUN_TIME)

Machine shop
Start parking at 0.00 hours
Parking for 6.18 hours
Start driving at 6.18 hours
Driving for 1.21 hours
Start parking at 7.40 hours
Parking for 15.16 hours
Start driving at 22.56 hours
Driving for 0.73 hours
Start parking at 23.28 hours
Parking for 6.55 hours
Start driving at 29.84 hours
Driving for 1.32 hours
Start parking at 31.16 hours
Parking for 4.16 hours
Start driving at 35.32 hours
Driving for 1.31 hours
Start parking at 36.64 hours
Parking for 1.91 hours
Start driving at 38.55 hours
Driving for 0.31 hours
Start parking at 38.85 hours
Parking for 9.43 hours
Start driving at 48.28 hours
Driving for 0.65 hours
Start parking at 48.93 hours
Parking for 19.07 hours
Start driving at 68.00 hours
Driving for 2.66 hours
Start parking at 70.66 hours
Parking for 99.63 hours


## Another Car Example
Modified from the example for the official simpy website: [Process Interaction](https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html)

In [3]:
def driver(env, car):
     yield env.timeout(3)
     car.action.interrupt()
    
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)
        
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


## Car Example with Resource

Modified from the example for the official simpy [website: Shared Resources](https://simpy.readthedocs.io/en/latest/simpy_intro/shared_resources.html)


In [5]:
def machine(env, name, crew, time_to_failure, repair_duration):
    while True:
    # Simulate time_to_failure
        yield env.timeout(time_to_failure)

        # Request one of its crew
        print('%s failure starts at %d' % (name, env.now))
        with crew.request() as req:
            yield req

            # Repair
            print('%s repair starts at %s' % (name, env.now))
            yield env.timeout(repair_duration)
            print('%s is repaired at %s' % (name, env.now))

import simpy
env = simpy.Environment()
crew = simpy.Resource(env, capacity=2)

for i in range(4):
    env.process(machine(env, 'Machine %d' % i, crew, i*2, 5))

env.run()

Machine 0 failure starts at 0
Machine 0 repair starts at 0
Machine 1 failure starts at 2
Machine 1 repair starts at 2
Machine 2 failure starts at 4
Machine 0 is repaired at 5
Machine 2 repair starts at 5
Machine 3 failure starts at 6
Machine 1 is repaired at 7
Machine 3 repair starts at 7
Machine 2 is repaired at 10
Machine 3 is repaired at 12


Add models for generating time to failure/repair
Repair: As good as new

In [13]:
class RandomTime:
    def __init__(self, list_para, distribution_type):
        self.distr = distribution_type
        self.para = list_para

    def time_to_event(self):
        if self.distr == 'Weibull':
            return random.weibullvariate(self.para[0], self.para[1]);
        elif self.distr == 'Lognormal':
            return random.weibullvariate(self.para[0], self.para[1]);
        else: 
            print('Unknown Distribution')
        
def machine(env, name, crew, failure_model, repair_model):
    while True:
        # Simulate time_to_failure
        yield env.timeout(failure_model.time_to_event())

        # Request one of its crew
        print('%s failure starts at %.2f' % (name, env.now))
        with crew.request() as req:
            yield req

            # Repair
            print('%s repair starts at %.2f' % (name, env.now))
            yield env.timeout(repair_model.time_to_event())
            print('%s is repaired at %.2f' % (name, env.now))

import simpy
env = simpy.Environment()
crew = simpy.Resource(env, capacity=2)
failure_model = RandomTime([30,1], "Weibull")
repair_model = RandomTime([5,1], "Lognormal")

for i in range(4):
    env.process(machine(env, 'Machine %d' % i, crew, failure_model, repair_model))

env.run(until = 100)

Machine 3 failure starts at 1.83
Machine 3 repair starts at 1.83
Machine 3 is repaired at 6.52
Machine 3 failure starts at 11.56
Machine 3 repair starts at 11.56
Machine 3 is repaired at 11.67
Machine 3 failure starts at 18.08
Machine 3 repair starts at 18.08
Machine 3 is repaired at 20.13
Machine 1 failure starts at 28.24
Machine 1 repair starts at 28.24
Machine 1 is repaired at 29.68
Machine 0 failure starts at 32.38
Machine 0 repair starts at 32.38
Machine 0 is repaired at 32.90
Machine 3 failure starts at 36.33
Machine 3 repair starts at 36.33
Machine 3 is repaired at 39.75
Machine 1 failure starts at 59.92
Machine 1 repair starts at 59.92
Machine 1 is repaired at 63.51
Machine 0 failure starts at 74.84
Machine 0 repair starts at 74.84
Machine 1 failure starts at 74.94
Machine 1 repair starts at 74.94
Machine 0 is repaired at 76.61
Machine 3 failure starts at 77.13
Machine 3 repair starts at 77.13
Machine 1 is repaired at 78.80
Machine 3 is repaired at 86.68
Machine 1 failure start

Add restoration factor


In [20]:
class RandomTime:
    # set current_time = 0, and restoration factor (rf) = 0 unless explicitly defined 
    def __init__(self, env, list_para, distribution_type, restoration_factor = 1):
        self.env = env
        self.distr = distribution_type
        self.para = list_para
        self.current_time = self.env.now
        self.rf = restoration_factor


    def time_to_event(self):
        if self.distr == 'Weibull':
            # calculate virtual time first
            v_time = self.current_time * (1 - self.rf)
            # conditional weibull random time to fail
            alpha = self.para[0]
            beta = self.para[1]
            ttf = alpha * (((v_time/alpha) ** beta - \
                     np.log(1-np.random.random_sample())) **(1. /beta)) - \
                     v_time + self.current_time
            print("time to next failure is %.2f" % ttf)
            return ttf;
        elif self.distr == 'Lognormal':
            return random.weibullvariate(self.para[0], self.para[1]);
        else: 
            print('Unknown Distribution')
        
def machine(env, name, crew, failure_model, repair_model):
    while True:
        # Simulate time_to_failure
        yield env.timeout(failure_model.time_to_event())

        # Request one of its crew
        print('%s failure starts at %.2f' % (name, env.now))
        with crew.request() as req:
            yield req

            # Repair
            print('%s repair starts at %.2f' % (name, env.now))
            yield env.timeout(repair_model.time_to_event())
            print('%s is repaired at %.2f' % (name, env.now))


env = simpy.Environment()
crew = simpy.Resource(env, capacity=2)
failure_model = RandomTime(env, [30,1], "Weibull", 0.5)
repair_model = RandomTime(env, [5,1], "Lognormal")
for i in range(4):
    env.process(machine(env, 'Machine %d' % i, crew, failure_model, repair_model))
env.run(until = 100)

time to next failure is 3.88
time to next failure is 12.61
time to next failure is 68.02
time to next failure is 21.92
Machine 0 failure starts at 3.88
Machine 0 repair starts at 3.88
Machine 0 is repaired at 9.24
time to next failure is 38.09
Machine 1 failure starts at 12.61
Machine 1 repair starts at 12.61
Machine 3 failure starts at 21.92
Machine 3 repair starts at 21.92
Machine 1 is repaired at 28.42
time to next failure is 104.45
Machine 3 is repaired at 32.29
time to next failure is 35.75
Machine 0 failure starts at 47.32
Machine 0 repair starts at 47.32
Machine 0 is repaired at 47.84
time to next failure is 8.94
Machine 0 failure starts at 56.78
Machine 0 repair starts at 56.78
Machine 0 is repaired at 58.07
time to next failure is 8.72
Machine 0 failure starts at 66.79
Machine 0 repair starts at 66.79
Machine 2 failure starts at 68.02
Machine 2 repair starts at 68.02
Machine 3 failure starts at 68.04
Machine 2 is repaired at 70.81
time to next failure is 23.96
Machine 3 repair

Add system log
As the code below, it is usable for simple RBD:
- All machines are running independently
- System down will not bring individual machine down. 
- System reliability metrics will be calculated from the event_log. 
- Restoration factor: 1 - "as-good-as-new", 0: "as-bad-as-old"

In [28]:
class EventLog:
    def __init__(self):
        self.event_time = []
        self.event_type = []
        self.event_machine_name = []

class RandomTime:
    # set current_time = 0, and restoration factor (rf) = 0 unless explicitly defined 
    def __init__(self, env, list_para, distribution_type, restoration_factor = 1):
        self.env = env
        self.distr = distribution_type
        self.para = list_para
        self.current_time = self.env.now
        self.rf = restoration_factor


    def time_to_event(self):
        if self.distr == 'Weibull':
            # calculate virtual time first
            v_time = self.current_time * (1 - self.rf)
            # conditional weibull random time to fail
            alpha = self.para[0]
            beta = self.para[1]
            ttf = alpha * (((v_time/alpha) ** beta - \
                     np.log(1-np.random.random_sample())) **(1. /beta)) - \
                     v_time
            print("time to next failure is %.2f" % ttf)
            return ttf;
        elif self.distr == 'Lognormal':
            return random.weibullvariate(self.para[0], self.para[1]);
        else: 
            print('Unknown Distribution')
        
def machine(env, name, crew, failure_model, repair_model, event_log):
    v_time = 0
    while True:
        # Simulate time_to_failure
        ttf = failure_model.time_to_event()
        yield env.timeout(ttf)
        # Request one of its crew
        print('%s failure starts at %.2f' % (name, env.now))
        event_log.event_time.append(env.now)
        event_log.event_type.append("down")
        event_log.event_machine_name.append(name)
        with crew.request() as req:
            yield req

            # Repair
            print('%s repair starts at %.2f' % (name, env.now))
            yield env.timeout(repair_model.time_to_event())
            print('%s is repaired at %.2f' % (name, env.now))
            event_log.event_time.append(env.now)
            event_log.event_type.append("up")
            event_log.event_machine_name.append(name)


env = simpy.Environment()
event_log = EventLog()
crew = simpy.Resource(env, capacity=2)
failure_model = RandomTime(env, [30,1], "Weibull", 0.5)
repair_model = RandomTime(env, [5,1], "Lognormal")
for i in range(4):
    env.process(machine(env, 'Machine %d' % i, crew, failure_model, repair_model, event_log))
env.run(until = 100)

time to next failure is 20.48
time to next failure is 3.65
time to next failure is 33.92
time to next failure is 78.89
Machine 1 failure starts at 3.65
Machine 1 repair starts at 3.65
Machine 1 is repaired at 4.84
time to next failure is 12.22
Machine 1 failure starts at 17.07
Machine 1 repair starts at 17.07
Machine 1 is repaired at 19.87
time to next failure is 17.04
Machine 0 failure starts at 20.48
Machine 0 repair starts at 20.48
Machine 0 is repaired at 22.59
time to next failure is 22.96
Machine 2 failure starts at 33.92
Machine 2 repair starts at 33.92
Machine 2 is repaired at 35.63
time to next failure is 41.01
Machine 1 failure starts at 36.91
Machine 1 repair starts at 36.91
Machine 0 failure starts at 45.54
Machine 0 repair starts at 45.54
Machine 0 is repaired at 47.66
time to next failure is 55.81
Machine 1 is repaired at 56.63
time to next failure is 10.39
Machine 1 failure starts at 67.02
Machine 1 repair starts at 67.02
Machine 1 is repaired at 67.30
time to next failu

In [26]:
print(event_log.event_time, "\n", event_log.event_type, "\n", event_log.event_machine_name)

[5.310712025548332, 7.2957733289668587, 12.585041660347915, 18.516376857515169, 23.989317361404844, 29.308940842140231, 31.868518295914377, 31.940594750363648, 53.810172950466281, 54.994527946452656, 77.010916333973768, 79.529758745681391] 
 ['down', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'up'] 
 ['Machine 2', 'Machine 1', 'Machine 1', 'Machine 1', 'Machine 2', 'Machine 2', 'Machine 1', 'Machine 2', 'Machine 0', 'Machine 0', 'Machine 0', 'Machine 0']
